In [3]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
from typing import List, Tuple
from preprocessing import clsf_boz_cd, clsf_rrnr_dvcd, clsf_dmfr_dvcd, clsf_crd_grd, get_cf, clsf_pdgr_cd, get_disc_factor

In [4]:
# 환경설정
pd.options.display.float_format = '{:,.0f}'.format
os.makedirs('result', exist_ok=True)

# 전역변수
FILE_PATH = Path('./data/현행추정부채_일반')

In [21]:
# 데이터 불러오기
## 익스포져
일반_원수_미경과보험료 = pd.read_excel(FILE_PATH / '일반_원수_미경과보험료.xlsx', dtype={'RRNR_DAT_DVCD': str, 'RRNR_CTC_BZ_DVCD': str, 'ARC_INPL_CD': str})
일반_출재_미경과보험료 = pd.read_excel(FILE_PATH / '일반_출재_미경과보험료.xlsx', dtype={'RRNR_DAT_DVCD': str, 'RRNR_CTC_BZ_DVCD': str, 'ARC_INPL_CD': str, 'T02_RN_RINSC_CD': str})
일반_원수_예정보험료 = pd.read_excel(FILE_PATH / '일반_원수_예정보험료.xlsx', dtype={'RRNR_DAT_DVCD': str, 'RRNR_CTC_BZ_DVCD': str, 'ARC_INPL_CD': str})
일반_출재_예정보험료 = pd.read_excel(FILE_PATH / '일반_출재_예정보험료.xlsx', dtype={'RRNR_DAT_DVCD': str, 'RRNR_CTC_BZ_DVCD': str, 'ARC_INPL_CD': str, 'T02_RN_RINSC_CD': str})
일반_원수_개별추산액 = pd.read_excel(FILE_PATH / '일반_원수_개별추산액.xlsx', dtype={'RRNR_DAT_DVCD': str, 'RRNR_DMFR_DVCD': str, 'ARC_INPL_CD': str})
일반_출재_개별추산액 = pd.read_excel(FILE_PATH / '일반_출재_개별추산액.xlsx', dtype={'RRNR_DAT_DVCD': str, 'RRNR_DMFR_DVCD': str, 'ARC_INPL_CD': str, 'T02_RN_RINSC_CD': str})

## 기초정보, 가정 등
일반_상품정보 = pd.read_excel(FILE_PATH / '일반_상품정보.xlsx', dtype={'PDC_CD': str, 'PDGR_CD': str})
일반_보험금진전추이 = pd.read_excel(FILE_PATH / '일반_보험금진전추이.xlsx')

## 공통
국가구분 = pd.read_excel(FILE_PATH / '국가구분.xlsx', dtype={'CNTR_CATG_CD': str})
재보험자_국내신용등급 = pd.read_excel(FILE_PATH / '재보험자_국내신용등급.xlsx', dtype={'재보험사코드': str}) \
    .rename(columns = {'재보험사코드': 'T02_RN_RINSC_CD', '국내신용등급': 'CRD_GRD'})
선도부도율 = pd.read_excel(FILE_PATH / '선도부도율.xlsx')
할인율 = pd.read_excel(FILE_PATH / '할인율.xlsx')

In [23]:
# 데이터 전처리
일반_원수_미경과보험료['RRNR_DVCD'] = clsf_rrnr_dvcd(일반_원수_미경과보험료, '원수')
일반_원수_미경과보험료['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_원수_미경과보험료)
일반_원수_미경과보험료['BOZ_CD'] = clsf_boz_cd(일반_원수_미경과보험료, 일반_상품정보)
일반_원수_미경과보험료['PDGR_CD'] = clsf_pdgr_cd(일반_원수_미경과보험료, 일반_상품정보)

일반_출재_미경과보험료['RRNR_DVCD'] = clsf_rrnr_dvcd(일반_출재_미경과보험료, '출재')
일반_출재_미경과보험료['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_출재_미경과보험료)
일반_출재_미경과보험료['BOZ_CD'] = clsf_boz_cd(일반_출재_미경과보험료, 일반_상품정보)
일반_출재_미경과보험료['PDGR_CD'] = clsf_pdgr_cd(일반_출재_미경과보험료, 일반_상품정보)
일반_출재_미경과보험료['CRD_GRD'] = clsf_crd_grd(일반_출재_미경과보험료, 재보험자_국내신용등급)

일반_원수_예정보험료['RRNR_DVCD'] = clsf_rrnr_dvcd(일반_원수_예정보험료, '원수')
일반_원수_예정보험료['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_원수_예정보험료)
일반_원수_예정보험료['BOZ_CD'] = clsf_boz_cd(일반_원수_예정보험료, 일반_상품정보)
일반_원수_예정보험료['PDGR_CD'] = clsf_pdgr_cd(일반_원수_예정보험료, 일반_상품정보)

일반_출재_예정보험료['RRNR_DVCD'] = clsf_rrnr_dvcd(일반_출재_예정보험료, '출재')
일반_출재_예정보험료['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_출재_예정보험료)
일반_출재_예정보험료['BOZ_CD'] = clsf_boz_cd(일반_출재_예정보험료, 일반_상품정보)
일반_출재_예정보험료['PDGR_CD'] = clsf_pdgr_cd(일반_출재_예정보험료, 일반_상품정보)
일반_출재_예정보험료['CRD_GRD'] = clsf_crd_grd(일반_출재_예정보험료, 재보험자_국내신용등급)

일반_원수_개별추산액['RRNR_DVCD'] = clsf_rrnr_dvcd(일반_원수_개별추산액, '원수')
일반_원수_개별추산액['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_원수_개별추산액)
일반_원수_개별추산액['BOZ_CD'] = clsf_boz_cd(일반_원수_개별추산액, 일반_상품정보)
일반_원수_개별추산액['PDGR_CD'] = clsf_pdgr_cd(일반_원수_개별추산액, 일반_상품정보)

일반_출재_개별추산액['RRNR_DVCD'] = clsf_rrnr_dvcd(일반_출재_개별추산액, '출재')
일반_출재_개별추산액['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_출재_개별추산액)
일반_출재_개별추산액['BOZ_CD'] = clsf_boz_cd(일반_출재_개별추산액, 일반_상품정보)
일반_출재_개별추산액['PDGR_CD'] = clsf_pdgr_cd(일반_출재_개별추산액, 일반_상품정보)
일반_출재_개별추산액['CRD_GRD'] = clsf_crd_grd(일반_출재_개별추산액, 재보험자_국내신용등급)

In [24]:
일반_출재_개별추산액

,RRNR_DAT_DVCD,RRNR_DMFR_DVCD,NTNL_CTRY_CD,ARC_INPL_CD,T02_RN_RINSC_CD,RN_LTPD_OST_AMT,RRNR_DVCD,DMFR_DVCD,BOZ_CD,PDGR_CD,CRD_GRD
0,01,1,KR,1040010,124012,4547133387,03,01,A001,23,AA
1,01,1,KR,1058410,124031,9000000,03,01,A007,28,AAA
2,01,1,KR,1069410,122086,42056520,03,01,A007,28,AA-
3,01,1,KR,1051810,124012,756784391,03,01,A003,27,AA
4,01,1,KR,1061710,185087,366260,03,01,A007,28,AA
...,...,...,...,...,...,...,...,...,...,...,...
1348,01,1,KR,1050611,123015,0,03,01,A003,27,AA+
1349,01,1,NaN,1050310,124035,350000,03,01,A003,27,AA+
1350,01,1,NaN,1050310,124096,700000,03,01,A003,27,AA+
1351,01,1,KR,1063811,999A21,0,03,01,A010,29,AA


In [58]:
# 데이터 집계
일반_원수_미경과보험료_집계 = 일반_원수_미경과보험료.groupby(['RRNR_DVCD', 'DMFR_DVCD', 'BOZ_CD'])['LTPD_URND_PRM'].sum().reset_index()
일반_원수_미경과보험료_집계.head()

,RRNR_DVCD,DMFR_DVCD,BOZ_CD,LTPD_URND_PRM
0,01,01,A001,23420916171
1,01,01,A002,27146263273
2,01,01,A003,96625029260
3,01,01,A004,22191701960
4,01,01,A005,5095840261


In [ ]:
# 데이터 내보내기